# Sumário

1. [Pré-Processamento](Pré)
2. [Machine Learning](ml)
3. [Gráficos](gf)



# Pré-Processamento dados Reais


In [1]:
import pandas as pd

In [2]:

df1 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df1.csv')# falha de subcarga
df2 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df2.csv')# sem falha de subcarga


In [3]:
df10 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df10.csv') # falha de subcarga 10%
df20 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df20.csv') # falha de subcarga 20%
df15 = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/df15.csv') # falha de subcarga 15%
dfbs = pd.read_csv('https://raw.githubusercontent.com/GUIPETAV/Base/main/dfbs.csv') #  sem de falha de subcarga

In [4]:
df2.head()

,Datetime,RTU_COMP_WATT,RTU_LA_COND_TEMP,RTU_MA_HUM,RTU_MA_TEMP,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,RTU_REFG_COND_TEMP_1,...,RTU_REFG_SUCT_PRES_2,RTU_REFG_SUCT_TEMP_1,RTU_REFG_SUCT_TEMP_2,RTU_SA_FAN_WATT,RTU_SA_FLOW,RTU_SA_HUM,RTU_SA_TEMP,RTU_TOT_WATT,ZA_HUM,ZA_TEMP
0,2020-07-31 00:01:00,0.0,23.181379,89.684395,21.856787,84.246849,22.704943,77.007149,23.505655,21.563417,...,8.048789,24.554947,24.686487,0.0,34.265842,76.671829,24.629072,41.000004,-14.803085,24.061129
1,2020-07-31 00:02:00,0.0,23.183462,89.673676,21.871866,84.245567,22.719912,77.201302,23.505655,21.569664,...,8.057360,24.550898,24.680414,0.0,34.274014,76.932068,24.629072,39.000004,-14.803085,24.062613
2,2020-07-31 00:03:00,0.0,23.216780,89.674660,21.874947,84.147728,22.741026,77.393867,23.505655,21.607147,...,8.065142,24.550901,24.670296,0.0,34.299866,77.198402,24.629072,40.000004,-14.803085,24.059637
3,2020-07-31 00:04:00,0.0,23.198038,89.653770,21.900015,84.074303,22.748741,77.490273,23.505655,21.640463,...,8.070160,24.587326,24.684464,0.0,34.271954,77.419304,24.629072,40.000004,-14.803085,24.059715
4,2020-07-31 00:05:00,0.0,23.162638,89.622345,21.908255,83.811562,22.763552,77.476524,23.505655,21.661285,...,8.076455,24.583281,24.668274,0.0,34.296387,77.423363,24.629072,40.000004,-14.803085,24.062654


In [5]:
# Remover as features incongruentes

columns_to_drop_real = ['RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',]
columns_to_drop_sim = ['RTU_TOT_CAPA','ZA_TEMP_SPT','RTU_OA_FLOW','RTU_SEN_CAPA','RTU_RA_FLOW','RTU_STG_STA','RTU_REFG_COND_TEMP']

In [6]:
df1_t = df1.assign(Target = 1) # falha de subcarga

df2_t = df2.assign(Target = 0) # não falha de subcarga


df20_t = df20.assign(Target = 1) # falha de subcarga

dfbs_t = dfbs.assign(Target = 0) #  não falha

In [7]:
# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df1_c = df1_t.loc[(df1_t['RTU_SA_FAN_WATT'] != 0) & (df1_t['RTU_COMP_WATT'] != 0)]
df2_c = df2_t.loc[(df2_t['RTU_SA_FAN_WATT'] != 0) & (df2_t['RTU_COMP_WATT'] != 0)]

# Eliminando  os momentos de desligameno do sistema no  site 2 de forma continua
# Seleciona as linhas que não contêm zero na FAN_WAAT e 'COMP_WATT'.

df20_c = df20_t.loc[(df20_t['RTU_COMP_WATT'] != 0) ]
dfbs_c = dfbs_t.loc[(dfbs_t['RTU_COMP_WATT'] != 0) ]


In [8]:
# Cria uma nova feature com a soma das pressões
df1_c = df1_c.assign(
    RTU_REFG_DISC_PRES=df1_c['RTU_REFG_DISC_PRES_1'] + df1_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df1_c['RTU_REFG_SUCT_PRES_1'] + df1_c['RTU_REFG_SUCT_PRES_2'])

df2_c = df2_c.assign(
    RTU_REFG_DISC_PRES=df2_c['RTU_REFG_DISC_PRES_1'] + df2_c['RTU_REFG_DISC_PRES_2'],
    RTU_REFG_SUCT_PRES=df2_c['RTU_REFG_SUCT_PRES_1'] + df2_c['RTU_REFG_SUCT_PRES_2'])


#### Padronizar a Pressão real

In [9]:
# Passo 1: Calcular a Derivada da Pressão (N-1) por Minuto

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
df1_c['Datetime'] = pd.to_datetime(df1_c['Datetime'])

df2_c['Datetime'] = pd.to_datetime(df2_c['Datetime'])

# Ordene o DataFrame pela coluna 'DataHora' (caso não esteja ordenado)
df1_c.sort_values(by='Datetime', inplace=True)

df2_c.sort_values(by='Datetime', inplace=True)


# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

df1_c['Derivada_RTU_REFG_DISC_PRES'] = df1_c['RTU_REFG_DISC_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()

df2_c['Derivada_RTU_REFG_DISC_PRES'] = df2_c['RTU_REFG_DISC_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()


# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'

df1_c['Derivada_RTU_REFG_SUCT_PRES'] = df1_c['RTU_REFG_SUCT_PRES'].diff() / df1_c['Datetime'].diff().dt.total_seconds()

df2_c['Derivada_RTU_REFG_SUCT_PRES'] = df2_c['RTU_REFG_SUCT_PRES'].diff() / df2_c['Datetime'].diff().dt.total_seconds()



In [10]:
# Passo 2 Calcular a Média móvel 


janela_media_movel = 1440   # Janela de um dia (1440 minutos)

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'

df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

df2_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df2_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'

df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

df2_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df2_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

In [11]:
# Passo 3 Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df1_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

df2_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df2_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'

df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df1_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()

df2_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df2_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()



In [12]:
# Passo 4 - Padronizar Manualmente

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
df1_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df1_c['Derivada_RTU_REFG_SUCT_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df2_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df2_c['Derivada_RTU_REFG_SUCT_PRES'] - df2_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df2_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df1_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df1_c['Derivada_RTU_REFG_DISC_PRES'] - df1_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df1_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

df2_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df2_c['Derivada_RTU_REFG_DISC_PRES'] - df2_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df2_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

In [13]:
df2_c.tail()

,Datetime,RTU_COMP_WATT,RTU_LA_COND_TEMP,RTU_MA_HUM,RTU_MA_TEMP,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,RTU_REFG_COND_TEMP_1,...,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES,DesvioPadrao_Derivada_RTU_REFG_DISC_PRES,DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_DISC_PRES
258579,2021-09-29 16:58:00,255.000015,18.664831,56.994507,19.833544,63.551861,8.283129,50.467957,23.729204,21.248987,...,15.702408,15.802378,-0.007623,0.011428,-0.003095,0.007251,4.763027,8.593972,0.000486,-0.000951
258580,2021-09-29 16:59:00,1983.000122,20.207827,56.758289,19.832773,62.563267,8.298266,50.420105,23.729204,21.051165,...,17.758046,13.318046,0.034261,-0.041406,-0.003072,0.007222,4.763027,8.593972,-0.005658,0.007838
258581,2021-09-29 17:00:00,445.000031,22.923174,57.022457,19.856422,62.345287,8.280314,50.353115,23.729204,18.766863,...,15.447086,14.390031,-0.038516,0.017866,-0.003099,0.007234,4.763027,8.593972,0.001237,-0.007436
258582,2021-09-29 17:01:00,3532.000000,22.515036,57.199764,19.847624,63.332333,8.304955,50.532452,23.729204,19.012575,...,18.055580,13.126932,0.043475,-0.021052,-0.003069,0.007219,4.763027,8.593972,-0.003290,0.009772
258583,2021-09-29 17:02:00,2766.000000,23.849806,57.171261,19.763884,62.924618,8.317967,50.466522,23.668657,16.809483,...,17.939455,10.044896,-0.001935,-0.051367,-0.003070,0.007184,4.763027,8.593972,-0.006813,0.000238


In [14]:
# Elimina as features real 
columns_to_drop = [
    'RTU_MA_HUM', 'RTU_MA_TEMP', 'RTU_LA_COND_TEMP',
    'RTU_REFG_DISC_PRES_1', 'RTU_REFG_DISC_PRES_2',
    'RTU_REFG_SUCT_PRES_1', 'RTU_REFG_SUCT_PRES_2',
    'RTU_REFG_COND_TEMP_1', 'RTU_REFG_COND_TEMP_2',
    'RTU_REFG_SUCT_TEMP_1', 'RTU_REFG_SUCT_TEMP_2',
    'RTU_REFG_DISC_TEMP_1', 'RTU_REFG_DISC_TEMP_2',
    'RTU_REFG_DISC_PRES', 'RTU_REFG_SUCT_PRES',
    'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES',
]

df1_f = df1_c.drop(columns_to_drop, axis=1)
df2_f = df2_c.drop(columns_to_drop, axis=1)



In [15]:
# Converter fahrenheit para celsius
def fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5/9
    return celsius

# Colunas a serem convertidas
columns_to_convert = ['RTU_RA_TEMP', 'RTU_OA_TEMP', 'ZA_TEMP', 'RTU_REFG_COND_TEMP',
                      'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_SA_TEMP']


# Aplicando a conversão para Celsius
df20[columns_to_convert] = df20[columns_to_convert].apply(fahrenheit_to_celsius)
dfbs[columns_to_convert] = dfbs[columns_to_convert].apply(fahrenheit_to_celsius)

#### Padronizar a Pressão 

In [16]:
# Converter colunas de Pascal para Bar
df20['RTU_REFG_SUCT_PRES'], df20['RTU_REFG_DISC_PRES'] = df20['RTU_REFG_SUCT_PRES'] /100000 , df20['RTU_REFG_DISC_PRES'] /100000
dfbs['RTU_REFG_SUCT_PRES'], dfbs['RTU_REFG_DISC_PRES'] = dfbs['RTU_REFG_SUCT_PRES'] / 100000, dfbs['RTU_REFG_DISC_PRES'] /100000

In [17]:
# Passo 1: Calcular a Derivada da Pressão (N-1) por Minuto

df20_c = df20_c.copy()
dfbs_c = dfbs_c.copy()

# Certifique-se de que as colunas de data/hora estão no formato de data/hora do pandas
df20_c['Datetime'] = pd.to_datetime(df20_c['Datetime'])

dfbs_c['Datetime'] = pd.to_datetime(dfbs_c['Datetime'])

# Ordene o DataFrame pela coluna 'DataHora' (caso não esteja ordenado)
df20_c.sort_values(by='Datetime', inplace=True)

dfbs_c.sort_values(by='Datetime', inplace=True)


# Calcule a derivada por minuto para 'RTU_REFG_DISC_PRES'

df20_c['Derivada_RTU_REFG_DISC_PRES'] = df20_c['RTU_REFG_DISC_PRES'].diff() / df20_c['Datetime'].diff().dt.total_seconds()

dfbs_c['Derivada_RTU_REFG_DISC_PRES'] = dfbs_c['RTU_REFG_DISC_PRES'].diff() / dfbs_c['Datetime'].diff().dt.total_seconds()


# Calcule a derivada por minuto para 'RTU_REFG_SUCT_PRES'
df20_c['Derivada_RTU_REFG_SUCT_PRES'] = df20_c['RTU_REFG_SUCT_PRES'].diff() / df20_c['Datetime'].diff().dt.total_seconds()

dfbs_c['Derivada_RTU_REFG_SUCT_PRES'] = dfbs_c['RTU_REFG_SUCT_PRES'].diff() / dfbs_c['Datetime'].diff().dt.total_seconds()



In [18]:
df20_c.head()

,Datetime,RTU_COMP_WATT,RTU_OA_FLOW,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_FLOW,RTU_RA_HUM,RTU_RA_TEMP,RTU_REFG_COND_PRES,RTU_REFG_COND_TEMP,...,RTU_SEN_CAPA,RTU_STG_STA,RTU_TOT_CAPA,RTU_TOT_WATT,ZA_HUM,ZA_TEMP,ZA_TEMP_SPT,Target,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES
0,2018-07-20 01:00:00,1.991743e+06,181.93083,58.777990,55.040030,2941.1106,59.538235,73.803910,24259538.0,77.66258,...,13232641.0,1.0,16450256.00,2.086172e+03,59.538235,73.803910,73.399994,1,NaN,NaN
1,2018-07-20 01:01:00,1.982156e+03,181.93083,58.826126,55.033990,2941.1106,59.586994,73.555954,24340444.0,77.88830,...,12469841.0,1.0,16648174.00,2.076584e+03,59.586994,73.555954,73.399994,1,1441.400000,2961.116667
2,2018-07-20 01:02:00,1.982326e+03,181.93083,58.870080,55028.000000,2941.1106,59.631520,73.310135,24327568.0,77.85254,...,12451516.0,1.0,16624992.00,2.076755e+03,59.631520,73.310135,73.399994,1,-206.933333,-282.033333
3,2018-07-20 01:03:00,1.983721e+03,181.93083,58.909492,55.022015,2941.1106,59.671448,73.066124,24310910.0,77.80606,...,12432533.0,1.0,16590.77,2.078149e+03,59.671448,73.066124,73.399994,1,-289.666667,-529.950000
4,2018-07-20 01:04:00,1.985772e+03,181.93083,58.944946,55.016030,2941.1106,59.707363,72.824150,24292254.0,77.75399,...,12413394.0,1.0,16550658.00,2.080201e+06,59.707363,72.824150,73.399994,1,-333.800000,-663.716667


In [19]:
# Passo 2 Calcular a Média móvel 


janela_media_movel = 1440   # Janela de um dia (1440 minutos)

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_DISC_PRES'
df20_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = df20_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

dfbs_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES'] = dfbs_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_media_movel).mean()

# Calcule a média móvel da derivada por minuto para 'RTU_REFG_SUCT_PRES'
df20_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = df20_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

dfbs_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES'] = dfbs_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_media_movel).mean()

In [20]:
df20_c.head()

,Datetime,RTU_COMP_WATT,RTU_OA_FLOW,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_FLOW,RTU_RA_HUM,RTU_RA_TEMP,RTU_REFG_COND_PRES,RTU_REFG_COND_TEMP,...,RTU_TOT_CAPA,RTU_TOT_WATT,ZA_HUM,ZA_TEMP,ZA_TEMP_SPT,Target,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES
0,2018-07-20 01:00:00,1.991743e+06,181.93083,58.777990,55.040030,2941.1106,59.538235,73.803910,24259538.0,77.66258,...,16450256.00,2.086172e+03,59.538235,73.803910,73.399994,1,NaN,NaN,NaN,NaN
1,2018-07-20 01:01:00,1.982156e+03,181.93083,58.826126,55.033990,2941.1106,59.586994,73.555954,24340444.0,77.88830,...,16648174.00,2.076584e+03,59.586994,73.555954,73.399994,1,1441.400000,2961.116667,NaN,NaN
2,2018-07-20 01:02:00,1.982326e+03,181.93083,58.870080,55028.000000,2941.1106,59.631520,73.310135,24327568.0,77.85254,...,16624992.00,2.076755e+03,59.631520,73.310135,73.399994,1,-206.933333,-282.033333,NaN,NaN
3,2018-07-20 01:03:00,1.983721e+03,181.93083,58.909492,55.022015,2941.1106,59.671448,73.066124,24310910.0,77.80606,...,16590.77,2.078149e+03,59.671448,73.066124,73.399994,1,-289.666667,-529.950000,NaN,NaN
4,2018-07-20 01:04:00,1.985772e+03,181.93083,58.944946,55.016030,2941.1106,59.707363,72.824150,24292254.0,77.75399,...,16550658.00,2.080201e+06,59.707363,72.824150,73.399994,1,-333.800000,-663.716667,NaN,NaN


In [21]:
# Passo 3 Calcular o Desvio Padrão


janela_um_dia = 1440 # Defina a janela de 1 dia (1440 minutos)


# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_DISC_PRES'
df20_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = df20_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()


dfbs_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES'] = dfbs_c['Derivada_RTU_REFG_DISC_PRES'].rolling(window=janela_um_dia).std()

# Calcule o desvio padrão da derivada por minuto dentro da janela de 1 dia para 'RTU_REFG_SUCT_PRES'

df20_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = df20_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()

dfbs_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES'] = dfbs_c['Derivada_RTU_REFG_SUCT_PRES'].rolling(window=janela_um_dia).std()


In [22]:
# Passo 4 - Padronizar Manualmente

# Calcule o Z-Score para cada minuto usando a média e o desvio padrão das colunas
df20_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (df20_c['Derivada_RTU_REFG_SUCT_PRES'] - df20_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / df20_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

dfbs_c['Z_Score_Derivada_RTU_REFG_SUCT_PRES'] = (dfbs_c['Derivada_RTU_REFG_SUCT_PRES'] - dfbs_c['MediaMovel_Derivada_RTU_REFG_SUCT_PRES']) / dfbs_c['DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES']

df20_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (df20_c['Derivada_RTU_REFG_DISC_PRES'] - df20_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / df20_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

dfbs_c['Z_Score_Derivada_RTU_REFG_DISC_PRES'] = (dfbs_c['Derivada_RTU_REFG_DISC_PRES'] - dfbs_c['MediaMovel_Derivada_RTU_REFG_DISC_PRES']) / dfbs_c['DesvioPadrao_Derivada_RTU_REFG_DISC_PRES']

In [23]:
# Elimina as features sim  

columns_to_drop2 = [
    'RTU_TOT_CAPA','ZA_TEMP_SPT','RTU_OA_FLOW','RTU_SEN_CAPA','RTU_RA_FLOW','RTU_STG_STA','RTU_REFG_COND_TEMP',
    'Derivada_RTU_REFG_DISC_PRES', 'Derivada_RTU_REFG_SUCT_PRES',
    'MediaMovel_Derivada_RTU_REFG_DISC_PRES','MediaMovel_Derivada_RTU_REFG_SUCT_PRES',
    'DesvioPadrao_Derivada_RTU_REFG_DISC_PRES','DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES',
]

dfbs_f = dfbs_c.drop(columns_to_drop2, axis=1)
df20_f = df20_c.drop(columns_to_drop2, axis=1)

### Definir X e y 

Real 

In [24]:
# eliminar os valores iniciais Nulos  das primeiras 24 hrs


# Para o DataFrame df1_x
df1_f = df1_c.dropna()

# Para o DataFrame df2_x
df2_f = df2_c.dropna()


In [25]:
# Definir y real

df1_y= df1_f['Target']
df2_y= df2_f['Target']

In [26]:
# Definir X real
df1_x= df1_f.drop(['Datetime','Target','RTU_REFG_SUCT_PRES_1', 'RTU_REFG_SUCT_PRES_2', 'RTU_REFG_DISC_TEMP_1', 'RTU_REFG_COND_TEMP_2', 'RTU_REFG_SUCT_TEMP_2', 'RTU_REFG_DISC_PRES_2', 'RTU_REFG_COND_TEMP_1', 'RTU_LA_COND_TEMP', 'RTU_MA_HUM', 'RTU_REFG_SUCT_TEMP_1', 'RTU_MA_TEMP', 'RTU_REFG_DISC_PRES_1', 'RTU_REFG_DISC_TEMP_2'], axis=1)
df2_x= df2_f.drop(['Datetime','Target','RTU_REFG_SUCT_PRES_1', 'RTU_REFG_SUCT_PRES_2', 'RTU_REFG_DISC_TEMP_1', 'RTU_REFG_COND_TEMP_2', 'RTU_REFG_SUCT_TEMP_2', 'RTU_REFG_DISC_PRES_2', 'RTU_REFG_COND_TEMP_1', 'RTU_LA_COND_TEMP', 'RTU_MA_HUM', 'RTU_REFG_SUCT_TEMP_1', 'RTU_MA_TEMP', 'RTU_REFG_DISC_PRES_1', 'RTU_REFG_DISC_TEMP_2'], axis=1)


SImulado 

In [27]:
# eliminar os valores iniciais zerados das primeiras 24 hrs

# Para o DataFrame df1_x
df20_f = df20_c.dropna()


# Para o DataFrame df2_x
dfbs_f= dfbs_c.dropna()


In [28]:
# Definir y sim

df20_y = df20_f['Target']
dfbs_y = dfbs_f['Target']

In [29]:
# Definir X sim

df20_x= df20_f.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP','RTU_RA_FLOW', 'RTU_OA_FLOW', 'RTU_STG_STA', 'RTU_SEN_CAPA', 'RTU_TOT_CAPA', 'RTU_REFG_COND_PRES', 'ZA_TEMP_SPT'], axis=1)
dfbs_x= dfbs_f.drop(['Datetime','Target', 'RTU_REFG_COND_TEMP', 'RTU_REFG_SUCT_TEMP', 'RTU_REFG_DISC_TEMP', 'RTU_RA_FLOW', 'RTU_OA_FLOW', 'RTU_STG_STA', 'RTU_SEN_CAPA', 'RTU_TOT_CAPA', 'RTU_REFG_COND_PRES', 'ZA_TEMP_SPT'], axis=1)

In [30]:


# Obtenha os nomes das colunas de cada DataFrame
nomes_df1= set(df2_x.columns)
nomes_dfbs = set(df20_x.columns)

# Encontre a diferença entre os nomes das colunas
colunas_exclusivas_df1 = nomes_df1 - nomes_dfbs
colunas_exclusivas_dfbs = nomes_dfbs - nomes_df1

# Imprima as colunas exclusivas em cada DataFrame
print("Colunas exclusivas em df1:", colunas_exclusivas_df1)
print("Colunas exclusivas em dfbs:", colunas_exclusivas_dfbs)

Colunas exclusivas em df1: set()
Colunas exclusivas em dfbs: set()


In [31]:
# Alinhar os Dataframes

# Obtenha a lista de colunas do df2_x
columns_to_keep = df2_x.columns.tolist()

# Garanta que todos os DataFrames tenham as mesmas colunas na mesma ordem
df1_x = df1_x[columns_to_keep]
dfbs_x = dfbs_x[columns_to_keep]
df20_x = df20_x[columns_to_keep]


In [32]:
df1_x.head()

,RTU_COMP_WATT,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,RTU_SA_FAN_WATT,RTU_SA_FLOW,RTU_SA_HUM,RTU_SA_TEMP,RTU_TOT_WATT,...,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES,DesvioPadrao_Derivada_RTU_REFG_DISC_PRES,DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_DISC_PRES
8020,4867.000488,61.200481,17.089087,74.737633,26.747133,1679.0,2697.217529,88.748802,15.716434,10636.000977,...,34.361689,10.115897,0.002609,0.000498,-0.001848,0.000447,15.691864,5.231601,0.000010,0.000284
8021,4848.000488,61.574734,17.105555,74.610580,26.805353,1679.0,2703.899658,88.873016,15.797943,10604.000977,...,34.186405,10.103673,-0.002921,-0.000204,-0.001860,0.000454,15.691864,5.231601,-0.000126,-0.000068
8022,4864.000000,61.070316,17.114769,74.525124,26.893845,1679.0,2703.134277,88.703865,15.797943,10633.000000,...,34.345150,10.105578,0.002646,0.000032,-0.001858,0.000467,15.691864,5.231601,-0.000083,0.000287
8023,4884.000488,60.565948,17.110941,74.531448,26.970688,1678.0,2688.689453,88.669090,15.797943,10665.000977,...,34.536175,10.125072,0.003184,0.000325,-0.001860,0.000468,15.691864,5.231601,-0.000027,0.000321
8024,4919.000000,61.987522,17.146467,74.556847,26.970688,1678.0,2667.598877,88.997284,15.818904,10724.000977,...,34.753290,10.157181,0.003619,0.000535,-0.001856,0.000465,15.691864,5.231601,0.000013,0.000349


In [33]:
dfbs_x.head()

,RTU_COMP_WATT,RTU_OA_HUM,RTU_OA_TEMP,RTU_RA_HUM,RTU_RA_TEMP,RTU_SA_FAN_WATT,RTU_SA_FLOW,RTU_SA_HUM,RTU_SA_TEMP,RTU_TOT_WATT,...,RTU_REFG_DISC_PRES,RTU_REFG_SUCT_PRES,Derivada_RTU_REFG_DISC_PRES,Derivada_RTU_REFG_SUCT_PRES,MediaMovel_Derivada_RTU_REFG_DISC_PRES,MediaMovel_Derivada_RTU_REFG_SUCT_PRES,DesvioPadrao_Derivada_RTU_REFG_DISC_PRES,DesvioPadrao_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_SUCT_PRES,Z_Score_Derivada_RTU_REFG_DISC_PRES
1473,2.081490e+03,53.366417,54.278023,54.056717,69.521385,94.42871,2941.1106,100.00000,48.346230,2.175919e+03,...,24307516.0,13396634.0,-314.566667,-808.116667,-317.598780,54.781681,14898.371423,12583.693099,-0.068573,0.000204
1474,1.073486e+06,53.049385,54.294994,53.736620,69.583786,66.10010,2058.7773,100.00000,47.670350,1.139586e+06,...,21975798.0,14206470.0,-38861.966667,13497.266667,-345.806350,61.825917,14932.853550,12588.379315,1.067289,-2.579290
1475,1.089443e+03,52.704006,54.312023,53.386760,69.681400,66.10010,2058.7773,100.00000,47.864037,1.155543e+03,...,21900688.0,13968659.0,-1251.833333,-3963.516667,-346.496651,59.376808,14932.872454,12588.819451,-0.319561,-0.060627
1476,1.093907e+03,52.370644,54.329163,53.049080,69.781710,66.10010,2058.7773,99.23971,48.028610,1.160007e+06,...,21882472.0,13905120.0,-303.600000,-1058.983333,-346.420123,59.147433,14932.872392,12588.836828,-0.088819,0.002868
1477,1.095146e+03,52.034534,54.346300,52.708614,69.884430,66.10010,2058.7773,98.67441,48.122658,1.161246e+03,...,21883398.0,13894214.0,15.433333,-181.766667,-346.079313,59.607028,14932.875048,12588.815929,-0.019174,0.024209


# cross validation real

In [34]:
# separar X falha
num_linhas = len(df1_x)
tamanho_parte = num_linhas // 5

# Separe o dataframe em cinco partes e criar uma lista
X1 = [df1_x.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

In [35]:
# separar y falha
num_linhas = len(df1_y)
tamanho_parte = num_linhas // 5

# Separe o dataframe em cinco partes e criar uma lista

y1 = [df1_y.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

In [36]:
#separar X nfalha

num_linhas = len(df2_x)
tamanho_parte = num_linhas // 5

# Separe o dataframe em cinco partes e criar uma lista
X2 = [df2_x.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

In [37]:
# separar y nfalha

num_linhas = len(df2_y)
tamanho_parte = num_linhas // 5

# Separe o dataframe em cinco partes e criar uma lista
y2 = [df2_y.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

In [38]:
# Combinações Possíveis
total_combinacoes = len(X1) * len(X2)
print("Total de combinações possíveis:", total_combinacoes)

Total de combinações possíveis: 25


In [39]:
combinacoes_X = []
combinacoes_y = []

for i1 in range(len(X1)):
    for i2 in range(len(X2)):
        X_teste = pd.concat([X1[i] for i in range(len(X1)) if i == i1] + [X2[j] for j in range(len(X2)) if j == i2])
        X_treino = pd.concat([X1[i] for i in range(len(X1)) if i != i1] + [X2[j] for j in range(len(X2)) if j != i2])
        combinacoes_X.append((X_teste, X_treino))

for i1 in range(len(y1)):
    for i2 in range(len(y2)):
        y_teste = pd.concat([y1[i] for i in range(len(y1)) if i == i1] + [y2[j] for j in range(len(y2)) if j == i2])
        y_treino = pd.concat([y1[i] for i in range(len(y1)) if i != i1] + [y2[j] for j in range(len(y2)) if j != i2])
        combinacoes_y.append((y_teste, y_treino))

# Cross simulado

In [40]:
# separar y nfalha

num_linhas = len(df20_y)
tamanho_parte = num_linhas // 5

num_linhas = len(dfbs_y)
tamanho_parte = num_linhas // 5


# Separe o dataframe em cinco partes e criar uma lista

y20 = [df20_y.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

ybs = [dfbs_y.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

In [41]:
total_combinacoes = len(y20) * len(ybs)
print("Total de combinações possíveis:", total_combinacoes)

Total de combinações possíveis: 25


In [42]:
#separar X nfalha

num_linhas = len(df20_x)
tamanho_parte = num_linhas // 5

num_linhas = len(dfbs_x)
tamanho_parte = num_linhas // 5

# Separe o dataframe em cinco partes e criar uma lista

X20 = [df20_x.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]

Xbs = [dfbs_x.iloc[(i*tamanho_parte):((i+1)*tamanho_parte)] for i in range(5)]


In [43]:
total_combinacoes = len(X20) * len(Xbs)
print("Total de combinações possíveis:", total_combinacoes)

Total de combinações possíveis: 25


In [44]:
combinacoes_Xs = []
combinacoes_ys = []

for i1 in range(len(X20)):
    for i2 in range(len(Xbs)):
        Xs_teste = pd.concat([X20[i] for i in range(len(X20)) if i == i1] + [Xbs[j] for j in range(len(Xbs)) if j == i2])
        Xs_treino = pd.concat([X20[i] for i in range(len(X20)) if i != i1] + [Xbs[j] for j in range(len(Xbs)) if j != i2])
        combinacoes_Xs.append((Xs_teste, Xs_treino))

for i1 in range(len(y20)):
    for i2 in range(len(ybs)):
        ys_teste = pd.concat([y20[i] for i in range(len(y20)) if i == i1] + [ybs[j] for j in range(len(ybs)) if j == i2])
        ys_treino = pd.concat([y20[i] for i in range(len(y20)) if i != i1] + [ybs[j] for j in range(len(ybs)) if j != i2])
        combinacoes_ys.append((ys_teste, ys_treino))

# Combinação Simulado Real

In [45]:
combinacoes_Xsr = []
combinacoes_ysr = []

for i1 in range(len(X20)):
    for i2 in range(len(Xbs)):
        Xsr_teste = pd.concat([X1[i] for i in range(len(X1)) if i == i1] + [X2[j] for j in range(len(X2)) if j == i2])
        Xsr_treino = pd.concat([X20[i] for i in range(len(X20)) if i != i1] + [Xbs[j] for j in range(len(Xbs)) if j != i2])
        combinacoes_Xsr.append((Xsr_teste, Xsr_treino))

for i1 in range(len(y20)):
    for i2 in range(len(ybs)):
        ysr_teste = pd.concat([y1[i] for i in range(len(y1)) if i == i1] + [y2[j] for j in range(len(y2)) if j == i2])
        ysr_treino = pd.concat([y20[i] for i in range(len(y20)) if i != i1] + [ybs[j] for j in range(len(ybs)) if j != i2])
        combinacoes_ysr.append((ysr_teste, ysr_treino))

# Modelos Preditivos(Classificadores)

In [46]:
import pandas as pd
import joblib
from sklearn.svm import SVC
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import (AdaBoostClassifier,
                              GradientBoostingClassifier,
                              ExtraTreesClassifier,
                              RandomForestClassifier)
from sklearn.preprocessing import StandardScaler

In [47]:
class ModeloAuxiliar(object):
    def __init__(self, clf, seed=27, params=None):
        if params:
            params['random_state'] = seed
            self.clf = clf(**params)
        else:
            self.clf = clf()

    def predict(self, x):
        return self.clf.predict(x)

    def fit(self, x, y):
        return self.clf.fit(x, y)

    def feature_importances(self, x, y):
        return self.clf.fit(x, y).feature_importances_

modelos = [
           {'nome': 'RForest',
            'modelo': RandomForestClassifier}
          ]

resultados = []
medias_acuracias = []
scaler = StandardScaler()

for i in range(len(combinacoes_Xsr)):
    X_teste, X_treino = combinacoes_Xsr[i]
    y_teste, y_treino = combinacoes_ysr[i]

    X_treino_scaled = scaler.fit_transform(X_treino)
    X_teste_scaled = scaler.transform(X_teste)

    acuracias = []

    for model in modelos:
        x = ModeloAuxiliar(clf=model['modelo'])

        # treinar o modelo
        x.fit(X_treino_scaled, y_treino)

        # gerar previsão
        y_pred = x.predict(X_teste_scaled)

        # calcular a acurácia
        acuracia = accuracy_score(y_teste, y_pred)

        acuracias.append(acuracia)

       # Calcula a acurácia balanceada

        acuracia_balanceada = balanced_accuracy_score(y_teste, y_pred)

        # Calcular a f1 balanceada

        f1_balanceada = f= f1_score(y_teste, y_pred, average='weighted')

        # calcular as demaism métricas
        precisao = precision_score(y_teste, y_pred)
        recall = recall_score(y_teste, y_pred)
        f1 = f1_score(y_teste, y_pred)
        matriz_confusao = confusion_matrix(y_teste, y_pred)

        resultados.append({'Nome': model['nome'],
                           'Acurácia': acuracia,
                           'Acurácia Balanceada': acuracia_balanceada,
                           'Precisão': precisao,
                           'Recall': recall,
                           'F1-score': f1,
                           'F1-score Balanceada': f1_balanceada,
                           'Matriz de Confusão': matriz_confusao,
                           'Predição': y_pred
                           })
        # Salvar o modelo treinado usando joblib
        model_filename = f"modelo_{model['nome']}_treinado)sim_realderiv.joblib"
        joblib.dump(x, model_filename)

In [48]:
modelos_df = pd.DataFrame(resultados)
modelos_df = modelos_df[['Nome', 'Acurácia','Acurácia Balanceada','Precisão', 'Recall', 'F1-score','F1-score Balanceada', 'Matriz de Confusão','Predição']]



modelos_df.to_csv('modelos_sim_realderiv.csv', index=False)

In [50]:
modelos_df.head(15)

,Nome,Acurácia,Acurácia Balanceada,Precisão,Recall,F1-score,F1-score Balanceada,Matriz de Confusão,Predição
0,RForest,0.091875,0.500000,0.091875,1.000000,0.168289,0.015462,"[[0, 20253], [0, 2049]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,RForest,0.095731,0.500588,0.091974,0.996584,0.168405,0.023771,"[[93, 20160], [7, 2042]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,RForest,0.091875,0.500000,0.091875,1.000000,0.168289,0.015462,"[[0, 20253], [0, 2049]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,RForest,0.095059,0.500656,0.091985,0.997560,0.168438,0.022264,"[[76, 20177], [5, 2044]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,RForest,0.082324,0.448023,0.083118,0.896047,0.152125,0.013977,"[[0, 20253], [213, 1836]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,RForest,0.907676,0.499753,0.000000,0.000000,0.000000,0.864175,"[[20243, 10], [2049, 0]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,RForest,0.812304,0.457332,0.020637,0.022450,0.021505,0.815834,"[[18070, 2183], [2003, 46]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,RForest,0.092458,0.495057,0.091048,0.988287,0.166735,0.018627,"[[37, 20216], [24, 2025]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,RForest,0.093624,0.470914,0.086953,0.933138,0.159081,0.030159,"[[176, 20077], [137, 1912]]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ..."
9,RForest,0.087705,0.223537,0.040161,0.389946,0.072822,0.099427,"[[1157, 19096], [1250, 799]]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
fig = px.box(df5, x='Nome', y='F1-score Balanceada', points='all', title='Boxplot da F1-Score Balanceada do Classificador Sim 0',color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_yaxes(range=[0, 1])
fig.show()